## Character-Level Text Generation with PyTorch


Building a character-level text generation model using a recurrent neural network (RNN) implemented with PyTorch. The model will be trained on a text corpus and used to generate new text character by character.

In [ ]:
import torch
import torch.nn as nn
import random

In [ ]:
# Data Preparation
# Load and preprocess the Shakespeare text
with open('Shakespeare complete.txt', 'r', encoding='utf-8') as file:
    text = file.read()

all_characters = sorted(set(text))
n_characters = len(all_characters)

# Create character to index and index to character dictionaries
char_to_index = {char: i for i, char in enumerate(all_characters)}
index_to_char = {i: char for i, char in enumerate(all_characters)}

#### Defining a character-level RNN model using PyTorch.

In [ ]:
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers):
        super(CharRNN, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, hidden):
        embedded = self.embedding(x)
        output, hidden = self.rnn(embedded, hidden)
        output = self.fc(output)
        return output, hidden


####  Training the model

In [ ]:
# Hyperparameters
input_size = n_characters
hidden_size = 256
output_size = n_characters
n_layers = 3
learning_rate = 0.005
seq_length = 100
temperature = 0.5

model = CharRNN(input_size, hidden_size, output_size, n_layers)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
n_epochs = 2000
print_every = 100

for epoch in range(n_epochs):
    start_idx = random.randint(0, len(text) - seq_length)
    end_idx = start_idx + seq_length + 1
    data = text[start_idx:end_idx]
    input_seq = torch.tensor([char_to_index[char] for char in data[:-1]])
    target_seq = torch.tensor([char_to_index[char] for char in data[1:]])
    
    optimizer.zero_grad()
    output, _ = model(input_seq.unsqueeze(0), None)
    loss = criterion(output.view(-1, output_size), target_seq)
    loss.backward()
    optimizer.step()
    
    if epoch % print_every == 0:
        print(f'Epoch [{epoch}/{n_epochs}], Loss: {loss:.4f}')

#### Text Generation step

In [ ]:
def generate_text(start_char, length=200, temperature=0.7):
    hidden = None
    input_char = torch.tensor([[char_to_index[start_char]]])
    generated_text = start_char
    
    for _ in range(length):
        output, hidden = model(input_char, hidden)
        output_dist = output.data.view(-1).div(temperature).exp()
        top_char_idx = torch.multinomial(output_dist, 1)[0]
        predicted_char = index_to_char[top_char_idx.item()]
        
        generated_text += predicted_char
        input_char = torch.tensor([[top_char_idx]])
    
    return generated_text

start_char = 'T'
generated_text = generate_text(start_char, length=1000, temperature=0.7)
print(generated_text)
